In [2]:
from kbsa.kbsa_main import run_kbsa, get_kbsa_datafile_name
import numpy as np
from utils import numeric_models as nm
from utils import plotter
from dolfin import Point
from kbsa.model_generators import get_CDR

In [3]:
models_list = ['cdr']
kernel_types_list = ['sobolev']
model_params_dict = {'cdr': {'mesh_2D_dir': 'data/CDR/mesh_save_dir/rectangle.xdmf', 
                            'mesh_steps': 0.025, 
                            't_end': 0.1, 
                            'num_steps':10, 
                            'return_bool': False,
                            'g_ineq_c': {'fuel': 0.02, 'oxygen': 0.14, 'product': 0.014, 'temp': 900}}}
x_domains_dict = {'cdr': np.array([[0.2,0.3],[0.01,0.2]])}
u_domains_dict = {'cdr': np.array([[5e11,1.5e12],[1.5e3,9.5e3],[850,1000],[200,400],[0.5,1.5]])}

In [4]:
from kbsa.utils_kbsa import _get_u_samples_arr_to_kernel, _get_u_samples_arr, _get_u_indexSuperset_oneHot
u_domain = u_domains_dict['cdr']
dim_of_U = len(u_domain)
u_indexSuperset_oneHot = _get_u_indexSuperset_oneHot(dim_of_U=dim_of_U, higher_order=False)
dim_of_u_superset = len(u_indexSuperset_oneHot)
u_indexSuperset_oneHot

['10000', '01000', '00100', '00010', '00001', '11111']

In [5]:
n = 1
u_samples_arr_to_kernel = _get_u_samples_arr_to_kernel(dim_of_u_superset=dim_of_u_superset, dim_of_U=dim_of_U, n=n)
u_samples_arr = _get_u_samples_arr(u_samples_arr_to_kernel=u_samples_arr_to_kernel, u_domain=u_domain, dim_of_U=dim_of_U,dim_of_u_superset=dim_of_u_superset, gamma_type='cdr')

In [6]:
u_test = u_samples_arr[0][0]
u_test

array([9.82731855e+11, 2.12989623e+03, 9.97660324e+02, 2.22426728e+02,
       8.01413092e-01])

In [7]:
my_fen = get_CDR(params=model_params_dict['cdr'])

In [8]:
result_fens = my_fen(u=u_test)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [9]:
x_0 = np.random.uniform(0,1)
x_1 = np.random.uniform(0,0.5)
print(f"{x_0}, {x_1}")

0.9102388738626014, 0.38420201662100795


In [10]:
result_fens

[Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 28), FiniteElement('Lagrange', triangle, 1)), 201),
 Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 28), FiniteElement('Lagrange', triangle, 1)), 205),
 Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 28), FiniteElement('Lagrange', triangle, 1)), 209),
 Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 28), FiniteElement('Lagrange', triangle, 1)), 213)]

In [11]:
from dolfin import File, XDMFFile, Constant, conditional, le, project
from dolfin import Mesh,FunctionSpace,Function

In [12]:
mesh = Mesh()
with XDMFFile("data/CDR/mesh_save_dir/rectangle.xdmf") as xdmf:
    xdmf.read(mesh)

In [13]:
with XDMFFile(mesh.mpi_comm(), 'data/CDR/state_saves/fuel_field') as xdmf_1:
    xdmf_1.write_checkpoint(result_fens[0], 'fuel_field', 0, XDMFFile.Encoding.HDF5, append=False)
with XDMFFile(mesh.mpi_comm(), 'data/CDR/state_saves/oxygen_field') as xdmf_2:
    xdmf_2.write_checkpoint(result_fens[1], 'oxygen_field', 0, XDMFFile.Encoding.HDF5, append=False)
with XDMFFile(mesh.mpi_comm(), 'data/CDR/state_saves/product_field') as xdmf_3:
    xdmf_3.write_checkpoint(result_fens[2], 'product_field', 0, XDMFFile.Encoding.HDF5, append=False)   
with XDMFFile(mesh.mpi_comm(), 'data/CDR/state_saves/temp_field') as xdmf_4:
    xdmf_4.write_checkpoint(result_fens[3], 'temp_field', 0, XDMFFile.Encoding.HDF5, append=False)

In [ ]:
V_1 = FunctionSpace(mesh, 'CG', 3) 
V_2 = FunctionSpace(mesh, 'CG', 3)
V_3 = FunctionSpace(mesh, 'CG', 3)
V_4 = FunctionSpace(mesh, 'CG', 3)
fuel_field_t_now = Function(V_1)
oxyxen_field_t_now = Function(V_2)
product_field_t_now = Function(V_3)
temp_field_t_now = Function(V_4)

with XDMFFile(mesh.mpi_comm(), 'data/CDR/state_saves/fuel_field') as xdmf_1:
    xdmf_1.read_checkpoint(fuel_field_t_now, 'fuel_field', 0)
with XDMFFile(mesh.mpi_comm(), 'data/CDR/state_saves/oxygen_field') as xdmf_2:
    xdmf_2.read_checkpoint(oxyxen_field_t_now, 'oxygen_field', 0)
with XDMFFile(mesh.mpi_comm(), 'data/CDR/state_saves/product_field') as xdmf_3:
    xdmf_3.read_checkpoint(product_field_t_now, 'product_field', 0) 
with XDMFFile(mesh.mpi_comm(), 'data/CDR/state_saves/temp_field') as xdmf_4:
    xdmf_4.read_checkpoint(temp_field_t_now, 'temp_field', 0)

In [ ]:
from dolfin import Point
print(fuel_field_t_now(Point([x_0, x_1])))
print(oxyxen_field_t_now(Point([x_0, x_1])))
print(product_field_t_now(Point([x_0, x_1])))
print(temp_field_t_now(Point([x_0, x_1])))

In [ ]:
from scipy import stats
a = np.random.uniform(0.0001,10)
b = np.random.uniform(10,10000)
logUnifs = stats.loguniform.rvs(a,b, size=100000)
np.sum(((np.log(logUnifs) - np.log(a))/(np.log(b)-np.log(a))>1)) + np.sum(((np.log(logUnifs) - np.log(a))/(np.log(b)-np.log(a))<0))

In [ ]:
x_0 = np.random.uniform(0,1)
x_1 = np.random.uniform(0,0.5)
[f(Point([x_0, x_1])) for f in result_fens]

In [ ]:
cdr_model = nm.model(model_type='cdr', 
                    t_end_cdr=0.5, 
                    num_steps_cdr=100, 
                    output_paraview=False, 
                    mesh_2D_dir='data/CDR/mesh_save_dir/rectangle.xdmf', 
                    mesh_steps=0.025)

In [ ]:
cdr_model.output_paraview = True
cdr_model.get_cdr(reset=True, t_end=0.5, num_steps=100)
cdr_model.save_curr_cdr_output_to_file()
plotter.plot(model=cdr_model, plot_type='2D_mesh')

In [ ]:
models_list = ['cdr']
kernel_types_list = ['sobolev']
model_params_dict = {'cdr': {'mesh_2D_dir': 'data/CDR/mesh_save_dir/rectangle.xdmf', 
                            'mesh_steps': 0.025, 
                            't_end': 0.1, 
                            'num_steps':10, 
                            'g_ineq_c': {'fuel': 0.02, 'oxygen': 0.14, 'product': 0.014, 'temp': 900}}}
x_domains_dict = {'cdr': np.array([[0.2,0.3],[0.01,0.2]])}
u_domains_dict = {'cdr': np.array([[5e11,1.5e12],[1.5e3,9.5e3],[850,1000],[200,400],[0.5,1.5]])}
num_of_trials = 20
# num_of_trials = 5
n_list = [500]
# n_list = [50]
hsic_types_dict = {'unbiased': False, 'biased': False, 'R2': False, 'fellman': True}


In [ ]:
for n_list in N_list_list:
        file_name = get_kbsa_datafile_name(models_list=models_list, 
                                x_domains_dict=x_domains_dict, 
                                kernel_types_list=kernel_types_list, 
                                hsic_types_dict=hsic_types_dict, 
                                n_list=n_list, 
                                num_of_trials=num_of_trials,
                                model_params_dict=model_params_dict)

        data = run_kbsa(hsic_types_dict=hsic_types_dict,
                        kernel_types_list=kernel_types_list,
                        models_list=models_list,
                        model_params_dict=model_params_dict,
                        x_domains_dict=x_domains_dict,
                        sample_x_from_mesh=True,
                        u_domains_dict=u_domains_dict,
                        num_of_trials=num_of_trials,
                        n_list=n_list,
                        m_same_as_n=True,
                        save_data=True,
                        with_gpu=False,
                        save_directory='data/g_1/kbsa',
                        file_name=file_name,
                        verbose_inner=True,
                        verbose_outer=True,
                        verbose_inner_inner=True)